In [1]:
import preprocess

In [2]:
tst = [2023, 8, 28, 'AM']
ted = [2023, 9, 6, 'AM']
region = ('CONUS', [-126.401171875,24.071240929282325,
                    -61.36210937500001,49.40003415463647])

In [3]:
import FireTime

list_of_ts = list(FireTime.t_generator(tst, ted))
list_of_ts

[[2023, 8, 28, 'AM'],
 [2023, 8, 28, 'PM'],
 [2023, 8, 29, 'AM'],
 [2023, 8, 29, 'PM'],
 [2023, 8, 30, 'AM'],
 [2023, 8, 30, 'PM'],
 [2023, 8, 31, 'AM'],
 [2023, 8, 31, 'PM'],
 [2023, 9, 1, 'AM'],
 [2023, 9, 1, 'PM'],
 [2023, 9, 2, 'AM'],
 [2023, 9, 2, 'PM'],
 [2023, 9, 3, 'AM'],
 [2023, 9, 3, 'PM'],
 [2023, 9, 4, 'AM'],
 [2023, 9, 4, 'PM'],
 [2023, 9, 5, 'AM'],
 [2023, 9, 5, 'PM'],
 [2023, 9, 6, 'AM']]

## Once per landcover file

If your landcover file is not in lat,lon then transform it and resave it. You will use this file a bunch later on.

In [4]:
preprocess.preprocess_landcover()

2024-02-12 13:51:53,077 - FireLog - INFO - func:preprocess_landcover took: 11.61 sec


## Once per region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [5]:
preprocess.preprocess_region(region)

2024-02-12 13:53:58,942 - FireLog - INFO - func:preprocess_region took: 2.01 min


## Once per input file

Next process each NRT file into half day files. First we'll get all the times that are of interest. This could also be done by inspecting all looking at all the files that exist and seeing which have not been preprocessed yet.

In [6]:
%%time
for sat in ["SNPP", "NOAA20"]:
    for t in list_of_ts[::2]:
        preprocess.preprocess_NRT_file(t, sat)

2024-02-12 13:53:58,947 - FireLog - INFO - preprocessing NRT file for 2023-8-28, SNPP
2024-02-12 13:53:59,517 - FireLog - INFO - func:preprocess_NRT_file took: 570.28 ms
2024-02-12 13:53:59,518 - FireLog - INFO - preprocessing NRT file for 2023-8-29, SNPP
2024-02-12 13:54:00,031 - FireLog - INFO - func:preprocess_NRT_file took: 512.55 ms
2024-02-12 13:54:00,033 - FireLog - INFO - preprocessing NRT file for 2023-8-30, SNPP
2024-02-12 13:54:00,507 - FireLog - INFO - func:preprocess_NRT_file took: 474.02 ms
2024-02-12 13:54:00,507 - FireLog - INFO - preprocessing NRT file for 2023-8-31, SNPP
2024-02-12 13:54:01,005 - FireLog - INFO - func:preprocess_NRT_file took: 497.25 ms
2024-02-12 13:54:01,007 - FireLog - INFO - preprocessing NRT file for 2023-9-1, SNPP
2024-02-12 13:54:01,647 - FireLog - INFO - func:preprocess_NRT_file took: 639.93 ms
2024-02-12 13:54:01,649 - FireLog - INFO - preprocessing NRT file for 2023-9-2, SNPP
2024-02-12 13:54:02,404 - FireLog - INFO - func:preprocess_NRT_fil

CPU times: user 11 s, sys: 272 ms, total: 11.3 s
Wall time: 11.3 s


## Once per region and t

Do initial filtering and clustering using the preprocessed region and the half day files.

Note: for the purpose of timing I am running the full month, but each of these steps could run in a separate proccess.

In [7]:
region = preprocess.read_region(region)

2024-02-12 13:54:10,547 - FireLog - INFO - func:read_region took: 286.84 ms


In [8]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, sensor="VIIRS", region=region)

2024-02-12 13:54:10,565 - FireLog - INFO - filtering and clustering 2023-8-28 AM, VIIRS, CONUS
2024-02-12 13:54:10,598 - FireLog - INFO - func:read_preprocessed took: 31.64 ms
2024-02-12 13:54:10,623 - FireLog - INFO - func:read_preprocessed took: 23.81 ms
2024-02-12 13:56:13,100 - FireLog - INFO - func:do_clustering took: 1.14 sec
2024-02-12 13:56:13,154 - FireLog - INFO - func:preprocess_region_t took: 2.04 min
2024-02-12 13:56:13,155 - FireLog - INFO - filtering and clustering 2023-8-28 PM, VIIRS, CONUS
2024-02-12 13:56:13,194 - FireLog - INFO - func:read_preprocessed took: 38.60 ms
2024-02-12 13:56:13,234 - FireLog - INFO - func:read_preprocessed took: 39.49 ms
2024-02-12 13:57:28,144 - FireLog - INFO - func:do_clustering took: 27.73 ms
2024-02-12 13:57:28,167 - FireLog - INFO - func:preprocess_region_t took: 1.25 min
2024-02-12 13:57:28,168 - FireLog - INFO - filtering and clustering 2023-8-29 AM, VIIRS, CONUS
2024-02-12 13:57:28,194 - FireLog - INFO - func:read_preprocessed took:

CPU times: user 16min 11s, sys: 928 ms, total: 16min 12s
Wall time: 16min 17s
